In [2]:
# Module 16 Challenge Deliverable 2: Determine Bias of Vine Reviews
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Loading in Amazon Reviews US Grocery dataset for analysis
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Grocery_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [37]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import count
from pyspark.sql.functions import round
from pyspark.sql.types import StructField, StringType, IntegerType, StructType
import pyspark.sql.functions as F
from pyspark.sql.functions import col, lit
from pyspark import sql as spark
from pyspark.sql.functions import rand
# Read in the Review dataset as a DataFrame
grocery_df = df
grocery_df.head(3)

[Row(marketplace='US', customer_id=42521656, review_id='R26MV8D0KG6QI6', product_id='B000SAQCWC', product_parent=159713740, product_title='The Cravings Place Chocolate Chunk Cookie Mix, 23-Ounce Bags (Pack of 6)', product_category='Grocery', star_rating=5, helpful_votes=0, total_votes=0, vine='N', verified_purchase='Y', review_headline='Using these for years - love them.', review_body="As a family allergic to wheat, dairy, eggs, nuts, and several other things, we love the entire Cravings Place line of products as it allows us to bake treats with minimal effort and ingredients. Most allergy-free and gluten-free mixes usually just omit one or two allergens at most, so it's great to see a mix created without many of the most common allergens. (Note these still have soy and corn). We consume these on a regular basis and have been doing so for years.", review_date='2015-08-31'),
 Row(marketplace='US', customer_id=12049833, review_id='R1OF8GP57AQ1A0', product_id='B00509LVIQ', product_parent=

In [6]:
# Create the vine_table DataFrame, df
vine_df = grocery_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26MV8D0KG6QI6|          5|            0|          0|   N|                Y|
|R1OF8GP57AQ1A0|          5|            0|          0|   N|                Y|
|R3VDC1QB6MC4ZZ|          5|            0|          0|   N|                N|
|R12FA3DCF8F9ER|          5|            0|          0|   N|                Y|
| RTWHVNV6X4CNJ|          5|            0|          0|   N|                Y|
| RIG9AWFOGRDVO|          2|            1|          1|   N|                Y|
|R1S1XSG4ZCHDGS|          5|            1|          1|   N|                Y|
| RB15NBVY5ELVW|          5|            2|          2|   N|                Y|
| R56358YM1ZJ7I|          5|            0|          0|   N|                N|
|R1ODXB3C9UP3NL|          1|            1|          3|   N|     

In [7]:
# Filter-1: Create a df where there are >= 20 total_votes  
vine20_df = vine_df.filter("total_votes>=20")
vine20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|
|R33VIWRD2X5IA9|          1|            0|         20|   N|                Y|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|
|R26UFXLLQ2N0CR|          1|            0|         20|   N|                Y|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|
|R2M6WNM2OZZHKS|          1|           20|         22|   N|     

In [ ]:
vine20_df.describe().show()

In [10]:
# Filter-2: Create a df where % of helpful_votes is >= 0.50 
vine_helpful50_df = vine20_df.withColumn('percentage', round(vine20_df['helpful_votes'] / vine20_df['total_votes'],2))
vine_helpful50_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percentage|
+--------------+-----------+-------------+-----------+----+-----------------+----------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|      0.91|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|      0.93|
|R33VIWRD2X5IA9|          1|            0|         20|   N|                Y|       0.0|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|      0.88|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|      0.76|
|R26UFXLLQ2N0CR|          1|            0|         20|   N|                Y|       0.0|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|      0.89|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|      0.88|
|R34UQPQCTSKQK9|     

In [11]:
vine_helpful50_df = vine_helpful50_df.filter(vine_helpful50_df["percentage"] >=0.5)
vine_helpful50_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percentage|
+--------------+-----------+-------------+-----------+----+-----------------+----------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|      0.91|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|      0.93|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|      0.88|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|      0.76|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|      0.89|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|      0.88|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|      0.88|
|R2M6WNM2OZZHKS|          1|           20|         22|   N|                N|      0.91|
| RO6JNMUJDE1LP|     

In [12]:
# Filter-3: Create a df where there is a Vine review. 
vine_Y_df = vine_helpful50_df.filter("cast(vine as string) == 'Y'")
vine_Y_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percentage|
+--------------+-----------+-------------+-----------+----+-----------------+----------+
|R3MKO875WGIEBJ|          5|           34|         37|   Y|                N|      0.92|
|R3OPNTK61FQ7MP|          5|           32|         37|   Y|                N|      0.86|
| RMDRU8I773X5U|          4|           22|         25|   Y|                N|      0.88|
| RR83RGWFEFZCL|          3|           61|         64|   Y|                N|      0.95|
|R2TMZ7GV8SPIRV|          4|           23|         23|   Y|                N|       1.0|
| RX88JLUIFA4Q2|          4|           84|         87|   Y|                Y|      0.97|
| RV3AHTDRHDH7R|          5|          253|        258|   Y|                N|      0.98|
|R13EGJI8C8YJ4Y|          5|           19|         20|   Y|                N|      0.95|
| R9SNSQTK4TJOG|     

In [13]:
# Filter-4: Create a df where there is Not a Vine review. 
vine_N_df = vine_helpful50_df.filter("cast(vine as string) == 'N'")
vine_N_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percentage|
+--------------+-----------+-------------+-----------+----+-----------------+----------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|      0.91|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|      0.93|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|      0.88|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|      0.76|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|      0.89|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|      0.88|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|      0.88|
|R2M6WNM2OZZHKS|          1|           20|         22|   N|                N|      0.91|
| RO6JNMUJDE1LP|     

In [14]:
# Count the total number of reviews, i.e. sum of the total_votes
totalVotes_sum = vine_helpful50_df.groupBy().agg(F.sum("total_votes")).withColumnRenamed("sum(total_votes)", "totalVotes_sum")
totalVotes_sum.show()

+--------------+
|totalVotes_sum|
+--------------+
|       1617847|
+--------------+



In [15]:
# Count the total number of 5-star reviews, i.e. star_rating == 5
vine_fivestar_df = vine_helpful50_df.filter("star_rating>=5")
vine_fivestar_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percentage|
+--------------+-----------+-------------+-----------+----+-----------------+----------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|      0.91|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|      0.88|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|      0.88|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|      0.88|
| RO6JNMUJDE1LP|          5|           37|         50|   N|                N|      0.74|
|R1WNL4OIMSKDBR|          5|           35|         36|   N|                Y|      0.97|
| R22M0N2OZSICN|          5|           21|         23|   N|                Y|      0.91|
|R1E43139TRHB5T|          5|           22|         22|   N|                Y|       1.0|
|R1L40ST6C2UJHK|     

In [16]:
# Total sum of 5-star reviews (votes) 
total5star_sum = vine_fivestar_df.groupBy().agg(F.sum("total_votes")).withColumnRenamed("sum(total_votes)", "total5star_sum")
total5star_sum.show()

+--------------+
|total5star_sum|
+--------------+
|        918209|
+--------------+



In [17]:
# Total of 5-star votes and is a Vine review; vine==Y
Y5star_df = vine_fivestar_df.filter("cast(vine as string) == 'Y'")
Y5star_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percentage|
+--------------+-----------+-------------+-----------+----+-----------------+----------+
|R3MKO875WGIEBJ|          5|           34|         37|   Y|                N|      0.92|
|R3OPNTK61FQ7MP|          5|           32|         37|   Y|                N|      0.86|
| RV3AHTDRHDH7R|          5|          253|        258|   Y|                N|      0.98|
|R13EGJI8C8YJ4Y|          5|           19|         20|   Y|                N|      0.95|
|R1A76AK493C8P8|          5|           83|         90|   Y|                N|      0.92|
|R1BRN47DX8UXA7|          5|          154|        159|   Y|                N|      0.97|
|R3V9P61EYXDNRV|          5|           51|         54|   Y|                N|      0.94|
|R2BDT05RK7TOJ9|          5|           21|         22|   Y|                N|      0.95|
|R1S9CD7RIIIBR3|     

In [18]:
Y5star_sum = Y5star_df.groupBy().agg(F.sum("total_votes")).withColumnRenamed("sum(total_votes)", "Y5star_sum")
Y5star_sum.show()

+----------+
|Y5star_sum|
+----------+
|      1084|
+----------+



In [19]:
# Total of 5-star votes and is Not a Vine review; vine==N
N5star_df = vine_fivestar_df.filter("cast(vine as string) == 'N'")
N5star_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percentage|
+--------------+-----------+-------------+-----------+----+-----------------+----------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|      0.91|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|      0.88|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|      0.88|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|      0.88|
| RO6JNMUJDE1LP|          5|           37|         50|   N|                N|      0.74|
|R1WNL4OIMSKDBR|          5|           35|         36|   N|                Y|      0.97|
| R22M0N2OZSICN|          5|           21|         23|   N|                Y|      0.91|
|R1E43139TRHB5T|          5|           22|         22|   N|                Y|       1.0|
|R1L40ST6C2UJHK|     

In [20]:
N5star_sum = N5star_df.groupBy().agg(F.sum("total_votes")).withColumnRenamed("sum(total_votes)", "N5star_sum")
N5star_sum.show()

+----------+
|N5star_sum|
+----------+
|    917125|
+----------+



In [59]:
# Joining the review or vote sums into one df.
joined1_df = totalVotes_sum.withColumn("index", lit(''))
joined2_df = total5star_sum.withColumn("index", lit(''))
joined3_df = Y5star_sum.withColumn("index", lit(''))
joined4_df = N5star_sum.withColumn("index", lit(''))
joined5_df = joined1_df.join(joined2_df, on="index", how="right")
joined6_df = joined5_df.join(joined3_df, on="index", how="right")
joined7_df = joined6_df.join(joined4_df, on="index", how="right")
joined7_df.show()

+-----+--------------+--------------+----------+----------+
|index|totalVotes_sum|total5star_sum|Y5star_sum|N5star_sum|
+-----+--------------+--------------+----------+----------+
|     |       1617847|        918209|      1084|    917125|
+-----+--------------+--------------+----------+----------+



In [57]:
# The percentage 5-star reviews calculated for all Vine reviews, vs ALL total votes.
Y5Star_percent = joined7_df.withColumn('percentage', round(joined7_df['Y5star_sum'] / joined7_df['totalVotes_sum'],6)).withColumnRenamed("percentage", "Y5star_percent")
Y5Star_percent.show()

+-----+--------------+--------------+----------+----------+--------------+
|index|totalVotes_sum|total5star_sum|Y5star_sum|N5star_sum|Y5star_percent|
+-----+--------------+--------------+----------+----------+--------------+
|     |       1617847|        918209|      1084|    917125|        6.7E-4|
+-----+--------------+--------------+----------+----------+--------------+



In [58]:
# The percentage 5-star reviews calculated for all non-Vine reviews, vs ALL total votes.
N5Star_percent = joined7_df.withColumn('percentage', round(joined7_df['N5star_sum'] / joined7_df['totalVotes_sum'],3)).withColumnRenamed("percentage", "N5star_percent")
N5Star_percent.show()

+-----+--------------+--------------+----------+----------+--------------+
|index|totalVotes_sum|total5star_sum|Y5star_sum|N5star_sum|N5star_percent|
+-----+--------------+--------------+----------+----------+--------------+
|     |       1617847|        918209|      1084|    917125|         0.567|
+-----+--------------+--------------+----------+----------+--------------+



In [60]:
# The percentage 5-star reviews calculated for all Vine reviews, vs total of all 5-star votes.
Y5Staronly_percent = joined7_df.withColumn('percentage', round(joined7_df['Y5star_sum'] / joined7_df['total5star_sum'],6)).withColumnRenamed("percentage", "Y5staronly_percent")
Y5Staronly_percent.show()

+-----+--------------+--------------+----------+----------+------------------+
|index|totalVotes_sum|total5star_sum|Y5star_sum|N5star_sum|Y5staronly_percent|
+-----+--------------+--------------+----------+----------+------------------+
|     |       1617847|        918209|      1084|    917125|          0.001181|
+-----+--------------+--------------+----------+----------+------------------+



In [61]:
# The percentage 5-star reviews calculated for all non-Vine reviews, vs total of all 5-star votes.
N5Staronly_percent = joined7_df.withColumn('percentage', round(joined7_df['N5star_sum'] / joined7_df['total5star_sum'],3)).withColumnRenamed("percentage", "N5staronly_percent")
N5Staronly_percent.show()

+-----+--------------+--------------+----------+----------+------------------+
|index|totalVotes_sum|total5star_sum|Y5star_sum|N5star_sum|N5staronly_percent|
+-----+--------------+--------------+----------+----------+------------------+
|     |       1617847|        918209|      1084|    917125|             0.999|
+-----+--------------+--------------+----------+----------+------------------+

